In [ ]:
!nvidia-smi
%cd /content/drive/MyDrive/Colab Notebooks/Cultivarium/

Wed Aug 13 07:07:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!unzip -q candy_data_06JAN25.zip -d custom_data

In [ ]:
!wget -O train_val_split.py https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py

# TO DO: Improve robustness of train_val_split.py script so it can handle nested data folders, etc
!python train_val_split.py --datapath="custom_data" --train_pct=0.9

--2025-08-13 07:08:26--  https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3203 (3.1K) [text/plain]
Saving to: ‘train_val_split.py’

train_val_split.py  100%[===================>]   3.13K  --.-KB/s    in 0.001s  

2025-08-13 07:08:27 (3.16 MB/s) - ‘train_val_split.py’ saved [3203/3203]

Created folder at /content/drive/MyDrive/Colab Notebooks/Cultivarium/data/train/images.
Created folder at /content/drive/MyDrive/Colab Notebooks/Cultivarium/data/train/labels.
Created folder at /content/drive/MyDrive/Colab Notebooks/Cultivarium/data/validation/images.
Created folder at /content/drive/MyDrive/Colab Notebooks/Cultivarium/data/validation/labels.


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [ ]:
# Python function to automatically create data.yaml config file
# 1. Reads "classes.txt" file to get list of class names
# 2. Creates data dictionary with correct paths to folders, number of classes, and names of classes
# 3. Writes data in YAML format to data.yaml

import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': 'data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = 'custom_data/classes.txt'
path_to_data_yaml = 'data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat data.yaml

Created config file at data.yaml

File contents:

path: data
train: train/images
val: validation/images
nc: 11
names:
- MMs_peanut
- MMs_regular
- airheads
- gummy_worms
- milky_way
- nerds
- skittles
- snickers
- starbust
- three_musketeers
- twizzlers


In [ ]:
!yolo detect train data=data.yaml model=yolo11s.pt epochs=60 imgsz=640

Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=Fal

In [ ]:
!yolo detect predict model=runs/detect/train2/weights/best.pt source=data/validation/images save=True

Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,417,057 parameters, 0 gradients, 21.3 GFLOPs

image 1/17 /content/drive/MyDrive/Colab Notebooks/Cultivarium/data/validation/images/158c744d-candy_123.jpg: 384x640 1 airheads, 84.0ms
image 2/17 /content/drive/MyDrive/Colab Notebooks/Cultivarium/data/validation/images/2e4d966d-candy_99.jpg: 480x640 2 airheadss, 1 starbust, 71.8ms
image 3/17 /content/drive/MyDrive/Colab Notebooks/Cultivarium/data/validation/images/3eace8b6-candy_49.jpg: 384x640 2 airheadss, 1 gummy_worms, 1 nerds, 1 three_musketeers, 12.3ms
image 4/17 /content/drive/MyDrive/Colab Notebooks/Cultivarium/data/validation/images/426f5963-candy_45.jpg: 384x640 1 MMs_peanut, 1 MMs_regular, 1 airheads, 2 milky_ways, 1 three_musketeers, 11.3ms
image 5/17 /content/drive/MyDrive/Colab Notebooks/Cultivarium/data/validation/images/73a058b9-candy_143.jpg: 384x640 1 milky_way, 1 starbust, 11.3ms
image 6/17 /content/d

In [ ]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'runs/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Create "candy_model" folder to store model weights and train results
#!mkdir candy_model
!cp runs/detect/train2/weights/best.pt candy_model/candy_model.pt
!cp -r runs/detect/train2 candy_model

# Zip into "candy_model.zip"
%cd candy_model
!zip candy_model.zip candy_model.pt
!zip -r candy_model.zip train2
#%cd /content

cp: cannot stat 'runs/detect/train2/weights/best.pt': No such file or directory
cp: cannot stat 'runs/detect/train2': No such file or directory
[Errno 2] No such file or directory: 'candy_model'
/content/drive/MyDrive/Colab Notebooks/Cultivarium/candy_model
updating: candy_model.pt (deflated 8%)
  adding: train2/ (stored 0%)
  adding: train2/weights/ (stored 0%)
  adding: train2/weights/last.pt (deflated 8%)
  adding: train2/weights/best.pt (deflated 8%)
  adding: train2/args.yaml (deflated 53%)
  adding: train2/labels_correlogram.jpg (deflated 42%)
  adding: train2/labels.jpg (deflated 30%)
  adding: train2/train_batch0.jpg (deflated 5%)
  adding: train2/train_batch1.jpg (deflated 5%)
  adding: train2/train_batch2.jpg (deflated 4%)
  adding: train2/results.csv (deflated 63%)
  adding: train2/train_batch500.jpg (deflated 9%)
  adding: train2/train_batch501.jpg (deflated 15%)
  adding: train2/train_batch502.jpg (deflated 11%)
  adding: train2/val_batch0_pred.jpg (deflated 9%)
  adding: 